In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
raw_df = keras.datasets.cifar10

In [4]:
(X_train, y_train), (X_test,y_test) = raw_df.load_data()

In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
y_train = y_train.reshape(-1,)

In [8]:
y_test = y_test.reshape(-1,)

In [9]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [10]:
X_train = X_train/255.0
X_test = X_test/255.0

In [11]:
X_train[0].shape

(32, 32, 3)

In [12]:
X_train = X_train.reshape(len(X_train), 32, 32, 3)
X_test = X_test.reshape(len(X_test), 32, 32, 3)

In [13]:
X_train[0].shape

(32, 32, 3)

In [14]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(32,32,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [15]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'output', project_name = 'FashionMNIST')

INFO:tensorflow:Reloading Oracle from existing project output/FashionMNIST/oracle.json


In [17]:
tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.10239999741315842

Best val_accuracy So Far: 0.5587999820709229
Total elapsed time: 00h 02m 59s
INFO:tensorflow:Oracle triggered exit


In [18]:
model = tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 80)        2240      
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 48)        96048     
                                                                 
 flatten (Flatten)           (None, 32448)             0         
                                                                 
 dense (Dense)               (None, 64)                2076736   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,175,674
Trainable params: 2,175,674
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, epochs=20, validation_split=0.1, initial_epoch=3)

Epoch 4/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.5369 - accuracy: 0.8102 - val_loss: 1.4061 - val_accuracy: 0.6064
Epoch 5/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.4552 - accuracy: 0.8371 - val_loss: 1.5973 - val_accuracy: 0.5886
Epoch 6/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.3780 - accuracy: 0.8673 - val_loss: 1.7695 - val_accuracy: 0.5834
Epoch 7/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.3056 - accuracy: 0.8947 - val_loss: 1.9254 - val_accuracy: 0.5810
Epoch 8/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.2478 - accuracy: 0.9129 - val_loss: 2.1237 - val_accuracy: 0.5782
Epoch 9/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.2083 - accuracy: 0.9284 - val_loss: 2.4762 - val_accuracy: 0.5766
Epoch 10/20
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1783 - accuracy: 0.9382 - val_loss: 2.6415 - val_accuracy